# Team 5 💣
1. Maximus Lee
2. Aloysius Woo
3. Lim Huai Fu
4. Tan Ai Xin
5. Jin Zhenglong

## Setup

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

### Install Environment
Est Time: 8 mins



In [ ]:
#@title Python

%cd /content
!sudo apt-get install python3.8
!sudo apt-get install python3.8-distutils

!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

In [ ]:
#@title GitHub Repos

!git clone https://github.com/AloysiusWooRY/FollowYourPose.git
!git clone https://github.com/open-mmlab/mmpose.git

In [ ]:
#@title FollowYourPose Requirements

%cd /content/FollowYourPose
%mkdir /content/FollowYourPose/checkpoints

# !export PYTHONPATH=/content/FollowYourPose:$PYTHONPATH
!export PYTHONPATH="${PYTHONPATH}:/usr/local/lib/python3.8/dist-packages:/usr/lib/python3.8/dist-packages"
!python3.8 -m pip install -q -U --pre triton==2.0.0 # to-do reinstall this, another requirements.txt overrides this
!apt update
!python3.8 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113
!python3.8 -m pip install -r requirements.txt

In [ ]:
#@title MMPose Requirements
# https://github.com/open-mmlab/mmpose
# https://blog.csdn.net/qq_21532607/article/details/130226728
# https://colab.research.google.com/github/open-mmlab/mmpose/blob/master/demo/MMPose_Tutorial.ipynb#scrollTo=aIEhiA44zw0y

%cd /content/mmpose
%mkdir video_input
%mkdir video_output
!python3.8 -m pip install -r requirements.txt

!python3.8 -m pip install -U openmim
!python3.8 -m mim install mmengine
!python3.8 -m mim install mmpose
!python3.8 -m mim install "mmcv>=2.0.0"
!python3.8 -m mim install "mmdet>=3.0.0"

!python3.8 -m pip install --upgrade --no-deps --force-reinstall accelerate

In [ ]:
#@title Others

import sys

path_to_append = '/usr/local/lib/python3.8/dist-packages'

# Check if the path is not already in sys.path before appending
if path_to_append not in sys.path:
    sys.path.append(path_to_append)

### Mount Drive 🗻

1. Run the first cell to authenicate
2. Click on the URL of the Failure in the output Failure("Error opening URL: ...")
3. Select the ICT3104 shared account
4. Once successful, you can close the tab
5. Run the next cell
6. Done

In [ ]:
# @title First Cell
%cd /content
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
# @title Next Cell
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/gdrive/MyDrive

### Symbolic Linking

In [ ]:
# Video selection
%mkdir /content/video/

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/video/* /content/video

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/video/* /content/video/

In [ ]:
# Checkpoints folder

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/checkpoints/* /content/FollowYourPose/checkpoints/

In [ ]:
# MMPose video in

# symbolic link edition
!ln -s /content/gdrive/MyDrive/ICT3104/video/MMPose/* /content/mmpose/video_input

# copy edition
# %cp -r /content/gdrive/MyDrive/ICT3104/video/MMPose/* /content/mmpose/video_input

## Video Browsing

In [ ]:
# @title Video Selection { display-mode: "form" }

import os
import ipywidgets as widgets
from IPython.display import display, HTML, Video, clear_output

directory_path = '/content/video'
folders = os.listdir(directory_path)
subdirectories = [content for content in folders if os.path.isdir(os.path.join(directory_path, content))]

# Create an output widget as a placeholder
output_placeholder = widgets.Output()
display(output_placeholder)

# Create the subdirectory dropdown
subdir_picker = widgets.Dropdown(
    options=subdirectories,
    value=None,
    description='Folder:',
    disabled=False,
)
display(subdir_picker)

# Create the video dropdown
video_picker = widgets.Dropdown(
    options=[],
    description='Video:',
    disabled=True,
)
display(video_picker)

# Create a button to display the selected video
display_button = widgets.Button(
    description="Display Video",
    disabled=True,
    layout=widgets.Layout(width='212px',margin='0px 0px 20px 90px')
)
display(display_button)

send_mmpose_button = widgets.Button(
    description="MMPose",
    disabled=True,
    layout=widgets.Layout(width='104px')
)

send_fyp_button = widgets.Button(
    description="FYP",
    disabled=True,
    layout=widgets.Layout(width='104px')
)

send_btn_hbox = widgets.HBox(
    [send_mmpose_button,send_fyp_button],
    layout=widgets.Layout(margin='-18px 0px 0px 88px')
)
display(send_btn_hbox)

# Function to update the video dropdown options based on the selected folder
def update_videos(change):
    selected_folder = change['new']
    directory_path = f"/content/video/{selected_folder}"
    video_options = [file for file in os.listdir(directory_path) if file.endswith('.mp4')]

    video_picker.options = video_options
    video_picker.disabled = False
    if not video_options:
        video_picker.value = None

# Link the subdirectory dropdown to the update function
subdir_picker.observe(update_videos, 'value')

# Function to display the selected video
def display_selected_video(change):
    selected_video = video_picker.value
    if selected_video:
        video_path = f"/content/video/{subdir_picker.value}/{selected_video}"
        video_display = Video(video_path, embed=True)

        # Clear the output placeholder and display the video
        with output_placeholder:
            clear_output()
            display(video_display)

# Link the button to the display function
display_button.on_click(display_selected_video)

# Function to enable the "Display Video" button when a video is selected
def enable_button(change):
    if video_picker.value:
        display_button.disabled = False
        send_fyp_button.disabled = False
        send_mmpose_button.disabled = False
    else:
        display_button.disabled = True
        send_fyp_button.disabled = True
        send_mmpose_button.disabled = True

# Link the video picker dropdown to the enable button function
video_picker.observe(enable_button, 'value')


Output()

Dropdown(description='Folder:', options=('Samples', 'Skeleton', 'Charades', 'MMPose', '.ipynb_checkpoints'), v…

Dropdown(description='Video:', disabled=True, options=(), value=None)

Button(description='Display Video', disabled=True, layout=Layout(margin='0px 0px 20px 90px', width='212px'), s…

## Inference: MMPOSE

In [ ]:
# @title Settings { display-mode: "form" }

from IPython.display import Markdown, display, clear_output
import ipywidgets as widgets


selected_video_input = widgets.Text(
    placeholder='Select a video above',
    description="Video:",
    disabled=True
)
display(selected_video_input)

mmpose_inf_button = widgets.Button(
    description="Start Inference",
    disabled=True,
)
button_margin = widgets.Layout(margin='0px 0px 20px 154px')
mmpose_inf_button.layout = button_margin
display(mmpose_inf_button)

def update_videos_inf(change):
    selected_video = video_picker.value

    selected_video_input.value = selected_video
    if selected_video:
        mmpose_inf_button.disabled = False
    else:
        mmpose_inf_button.disabled = True

send_mmpose_button.on_click(update_videos_inf)

def run_mmpose_inference(button):
    selected_video = video_picker.value
    video_path = f"../video/{subdir_picker.value}/{selected_video}"

    %cd /content/mmpose

    output_dir = "/content/video/Skeleton"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Start inference
    !python demo/inferencer_demo.py \
        {video_path}  \
        --pose2d human --vis-out-dir {output_dir} \
        --black-background

    # Re-encode video due to H.264 video encoding error
    input_file = f"{output_dir}/{selected_video}"
    temp_output_file = f"{output_dir}/output.mp4"
    !ffmpeg -i {input_file} -c:v libx264 -crf 23 -c:a aac -strict experimental {temp_output_file}

    os.remove(input_file)
    os.rename(temp_output_file, input_file)

    clear_output()
    print(f"Done [Output: {input_file}]")

# Link the button to the print_selections function
mmpose_inf_button.on_click(run_mmpose_inference)


Text(value='', description='Video:', disabled=True, placeholder='Select a video above')

Button(description='Start Inference', disabled=True, layout=Layout(margin='0px 0px 20px 154px'), style=ButtonS…

**TODO**:
1. ~~H.264 video encoding error, re-encode video using FFmpeg~~
2. Add textbox for user to enter output folder path

## Inference: FYP

In [ ]:
# @title Config { display-mode: "form" }
from omegaconf import OmegaConf

!cd /content/FollowYourPose
sample_yaml_path = "./configs/pose_sample.yaml"
load_config = OmegaConf.load(sample_yaml_path)

config_style = {'description_width': '150px'}
config_layout = widgets.Layout(width="500px")

config_pretrained_model_path = widgets.Text(description="pretrained_model_path:", value=load_config.pretrained_model_path, style=config_style, layout=config_layout)
config_output_dir = widgets.Text(description="output_dir:", value=load_config.output_dir, style=config_style, layout=config_layout)
config_train_batch_size = widgets.IntText(description="train_batch_size:", value=load_config.train_batch_size, style=config_style, layout=config_layout)
config_validation_steps = widgets.IntText(description="validation_steps:", value=load_config.validation_steps, style=config_style, layout=config_layout)
config_resume_from_checkpoint = widgets.Text(description="resume_from_checkpoint:", value=load_config.resume_from_checkpoint, style=config_style, layout=config_layout)
config_seed = widgets.IntText(description="seed:", value=load_config.seed, style=config_style, layout=config_layout)
config_vbox = widgets.VBox([
    config_pretrained_model_path,
    config_output_dir,
    config_train_batch_size,
    config_validation_steps,
    config_resume_from_checkpoint,
    config_seed
])
display(config_vbox)

display(widgets.HTML(value="<h3>Validation Data</h3>"))
config_prompts = widgets.Textarea(description="prompts:", value="\n".join(load_config.validation_data.prompts), style=config_style, layout=widgets.Layout(width="500px", height="100px"))
config_video_length = widgets.IntText(description="video_length:", value=load_config.validation_data.video_length, style=config_style, layout=config_layout)
config_width = widgets.IntText(description="width:", value=load_config.validation_data.width, style=config_style, layout=config_layout)
config_height = widgets.IntText(description="height:", value=load_config.validation_data.height, style=config_style, layout=config_layout)
config_num_inference_steps = widgets.IntText(description="num_inference_steps:", value=load_config.validation_data.num_inference_steps, style=config_style, layout=config_layout)
config_guidance_scale = widgets.FloatText(description="guidance_scale:", value=load_config.validation_data.guidance_scale, style=config_style, layout=config_layout)
config_num_inv_steps = widgets.IntText(description="num_inv_steps:", value=load_config.validation_data.num_inv_steps, style=config_style, layout=config_layout)
config_num_dataset_set = widgets.Text(description="dataset_set:", value=load_config.validation_data.dataset_set, style=config_style, layout=config_layout)

config_vbox_validation_data = widgets.VBox([
    config_prompts,
    config_video_length,
    config_width,
    config_height,
    config_num_inference_steps,
    config_guidance_scale,
    config_num_inv_steps,
    config_num_dataset_set
])
display(config_vbox_validation_data)

config_save_btn = widgets.Button(description="Save")
config_save_btn.layout = widgets.Layout(margin='0px 0px 20px 354px')
display(config_save_btn)

def save_config(change):
  config = {
      "pretrained_model_path": config_pretrained_model_path.value,
      "output_dir": config_output_dir.value,
      "validation_data": {
          "prompts": [prompt.strip() for prompt in config_prompts.value.splitlines() if prompt.strip()],
          "video_length": config_video_length.value,
          "width": config_width.value,
          "height": config_height.value,
          "num_inference_steps": config_num_inference_steps.value,
          "guidance_scale": config_guidance_scale.value,
          "use_inv_latent": load_config.validation_data.use_inv_latent,
          "num_inv_steps": config_num_inv_steps.value,
          "dataset_set": config_num_dataset_set.value
      },
      "train_batch_size": config_train_batch_size.value,
      "validation_steps": config_validation_steps.value,
      "resume_from_checkpoint": config_resume_from_checkpoint.value,
      "seed": config_seed.value,
      "mixed_precision": load_config.mixed_precision,
      "gradient_checkpointing": load_config.gradient_checkpointing,
      "enable_xformers_memory_efficient_attention": load_config.enable_xformers_memory_efficient_attention
  }
  OmegaConf.save(config, sample_yaml_path)
  print("Successfully saved!")

config_save_btn.on_click(save_config)



In [ ]:
# @title Settings { display-mode: "form" }

skeleton_file_path = widgets.Text(
    placeholder='Select a video above',
    description="Video:",
    disabled=True
)
display(skeleton_file_path)

config_file_path = widgets.Text(
    value='configs/pose_sample.yaml',
    description="Config File:"
)
display(config_file_path)

fyp_inf_button = widgets.Button(
    description="Start Inference",
    disabled=True,
)
button_margin = widgets.Layout(margin='0px 0px 20px 154px')
fyp_inf_button.layout = button_margin
display(fyp_inf_button)

def update_skeleton_inf(change):
    selected_video = video_picker.value
    skeleton_file_path.value = f"../video/{subdir_picker.value}/{selected_video}"

    if selected_video:
        fyp_inf_button.disabled = False
    else:
        fyp_inf_button.disabled = True

send_fyp_button.on_click(update_skeleton_inf)

def run_fyp_inference(button):
    config_file_path_text = config_file_path.value
    video_file_path_text = skeleton_file_path.value

    !cd /content/FollowYourPose

    # Start inference
    !TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py \
        --config={config_file_path_text}  \
        --skeleton_path={video_file_path_text}

# Link the button to the print_selections function
fyp_inf_button.on_click(run_fyp_inference)


Text(value='', description='Video:', disabled=True, placeholder='Select a video above')

Text(value='configs/pose_sample.yaml', description='Config File:')

Button(description='Start Inference', disabled=True, layout=Layout(margin='0px 0px 20px 154px'), style=ButtonS…

## Training (WIP)

In [ ]:

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the base directory path
base_directory = '/content'

# Function to create nested dropdowns for directories
def create_directory_dropdown(directory_path, default_value='Select'):
    folder_picker = widgets.Dropdown(
        options=['Select'] + get_subdirectories(directory_path),
        value=default_value,
        description=os.path.basename(directory_path) + ':',
        disabled=False,
    )
    folder_picker.style.description_width = 'initial'  # Allow the description to shrink
    folder_picker.layout.width = 'auto'  # Allow the dropdown to adjust its width
    folder_picker.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    folder_picker.layout.width = '400px'
    return folder_picker

# Function to create "Training Epochs" textbox
def create_epochs_textbox(default_value=10):
    epochs_textbox = widgets.IntText(
        value=default_value,
        description='Training Epochs:',
        layout=widgets.Layout(width='auto')
    )
    epochs_textbox.style.description_width = 'initial'  # Allow the description to shrink
    epochs_textbox.layout.width = 'auto'  # Allow the textbox to adjust its width
    epochs_textbox.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    epochs_textbox.layout.width = '400px'
    return epochs_textbox

# Function to create "Batch Size" textbox
def create_batch_size_textbox(default_value=32):
    batch_size_textbox = widgets.IntText(
        value=default_value,
        description='Batch Size:',
        layout=widgets.Layout(width='auto')
    )
    batch_size_textbox.style.description_width = 'initial'  # Allow the description to shrink
    batch_size_textbox.layout.width = 'auto'  # Allow the textbox to adjust its width
    batch_size_textbox.layout.min_width = '200px'  # Set a minimum width to ensure content is visible
    batch_size_textbox.layout.width = '400px'
    return batch_size_textbox

# Function to create "Submit" button
def create_submit_button():
    submit_button = widgets.Button(
        description='Submit',
    )
    submit_button.layout.margin = '20px 0 0 200px'  # Add margin for positioning
    return submit_button

# Function to get subdirectories in a folder
def get_subdirectories(folder_path):
    folders = os.listdir(folder_path)
    return [content for content in folders if os.path.isdir(os.path.join(folder_path, content))]

# Function to display the selected values with full paths
def display_selected_values(selected_folder, subfolder, epochs, batch_size):
    clear_output(wait=True)
    dataset_folder_path = os.path.join(base_directory, selected_folder)
    subfolder_path = os.path.join(dataset_folder_path, subfolder) if subfolder else None
    print(f'Selected Dataset Folder: {dataset_folder_path}')
    if subfolder_path:
        print(f'Selected Dataset Subfolder: {subfolder_path}')
    print(f'Epochs Entered: {epochs}')
    print(f'Batch Size Entered: {batch_size}')

# Create an output widget as a placeholder
output_placeholder = widgets.Output()

# Create the "Dataset Folder" dropdown
dataset_folder_picker = create_directory_dropdown(base_directory)
dataset_folder_picker.description = 'Dataset Folder:'

# Create empty variables for the subfolder, epochs, and batch size widgets
subfolder_picker = None
epochs_textbox = None
batch_size_textbox = None
submit_button = None

# Function to handle button click
def on_submit_button_click(b):
    selected_folder = dataset_folder_picker.value
    subfolder = subfolder_picker.value if subfolder_picker else None
    epochs = epochs_textbox.value
    batch_size = batch_size_textbox.value
    display_selected_values(selected_folder, subfolder, epochs, batch_size)

# Function to update the directory dropdowns based on the selected "Dataset Folder"
def update_directory_dropdowns(change):
    global subfolder_picker, epochs_textbox, batch_size_textbox, submit_button

    selected_folder = change['new']

    # Clear previous subfolder, epochs, and batch size widgets
    if subfolder_picker:
        subfolder_picker.close()
    if epochs_textbox:
        epochs_textbox.close()
    if batch_size_textbox:
        batch_size_textbox.close()
    if submit_button:
        submit_button.close()

    # Create the "Dataset Subfolder" dropdown if a valid folder is selected
    if selected_folder != 'Select':
        subfolder_picker = create_directory_dropdown(os.path.join(base_directory, selected_folder), default_value='Select')
        subfolder_picker.description = 'Dataset Subfolder:'
        display(subfolder_picker)

        # Create "Training Epochs" and "Batch Size" textboxes
        epochs_textbox = create_epochs_textbox()
        batch_size_textbox = create_batch_size_textbox()
        submit_button = create_submit_button()
        submit_button.on_click(on_submit_button_click)

        # Display the "Submit" button
        display(epochs_textbox, batch_size_textbox, submit_button)

# Observe changes in the "Dataset Folder" dropdown
dataset_folder_picker.observe(update_directory_dropdowns, names='value')

# Display the widgets
display(dataset_folder_picker, output_placeholder)
